In [1]:
import numpy as np
from skimage.draw import disk
import tensorstore_trackarray as tta

def circular_blob_image(frame_count, image_size, num_blobs, blob_diameter, dtype):
    image = np.zeros((frame_count, image_size, image_size), dtype=dtype)
    radius = blob_diameter // 2
    for frame in range(frame_count):
        for i in range(num_blobs):
            # Randomly select the center of the circle
            center_x = np.random.randint(radius, image.shape[1] - radius)
            center_y = np.random.randint(radius, image.shape[2] - radius)

            # Get the coordinates of the circle
            rr, cc = disk((center_x, center_y), radius, shape=image.shape[1:])

            # Draw the circle
            image[frame, rr, cc] = i
    return image 
labels = circular_blob_image(10, 8192*2, 3000, 30, np.uint16)
bboxes_df = tta.to_bbox_df(labels)

In [2]:
bboxes_df.to_csv('bboxes.csv', index=False)

In [9]:
%timeit bboxes_df[bboxes_df.index.get_level_values('label') == 1].reset_index()

413 μs ± 14.9 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [7]:
bboxes_df2=bboxes_df.reset_index().set_index(["label","frame"])

In [10]:
%timeit bboxes_df2.loc[1]

117 μs ± 4.88 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [11]:
bboxes_data = {label:grp for label, grp in bboxes_df.reset_index().groupby("label")}

In [12]:
%timeit bboxes_data[1]

49.8 ns ± 21.5 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [ ]:
def get_bboxes(label):
    